In [20]:
!mkdir -p ~/agave

%cd ~/agave

!pip3 install --upgrade setvar

import re
import os
import sys
from setvar import *
from time import sleep

# This cell enables inline plotting in the notebook
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import runagavecmd as r
!auth-tokens-refresh

/home/jovyan/agave
Requirement already up-to-date: setvar in /opt/conda/lib/python3.6/site-packages
Token for agave.prod:stevenrbrandt successfully refreshed and cached for 14400 seconds
3b31b4863a6ca37b99af7834598f5196


<h2>A small Gateway</h2>
Jupyter gives you the ability to use "widgets" to create a more traditional interactive GUI. Normally, one would move all the python code below into a module so that the page is less cluttered. However, this is a tutorial so we'll leave it in.

In [36]:
# import necessary libraries
from numpy import *
#import scipy
from scipy import optimize
from pylab import *
from ipywidgets import Button, Text, Label, Box, Layout, BoundedFloatText
from IPython.display import clear_output
import matplotlib.pyplot as plt

waveAmplitude = BoundedFloatText(value=3,min=0,max=5,step=.1)
waveWidth = BoundedFloatText(value=25,min=1,max=50,step=.1)
waveXc = BoundedFloatText(value=50,min=0,max=500,step=.1)
waveYc = BoundedFloatText(value=20,min=0,max=100,step=.1)
plotBtn = Button(description='Generate input.txt')
runBtn = Button(description="Submit Job")

input_layout = Layout(
    flex_flow = 'column',
    align_items = 'stretch',
    disabled = False
)
label_layout = Layout(
    width = '130px'
)
text_layout = Layout( 
    display = 'flex',
    flex_flow = 'row',
    justify_content = 'flex-start',
    width = '50%'
)
inputBox = Box(
    [
        Box([Label(value='Wave Amplitude: ',layout = label_layout), waveAmplitude], layout = text_layout),
        Box([Label(value='Wave Width: ',layout = label_layout), waveWidth], layout = text_layout),
        Box([Label(value='waveXc: ',layout = label_layout), waveXc], layout = text_layout), 
        Box([Label(value='waveYc: ',layout = label_layout), waveYc], layout = text_layout), 
        Box([plotBtn],layout = text_layout),
        Box([runBtn],layout = text_layout)
    ], 
    layout = input_layout
)
def runBtn_clicked(a):
    os.system(repvar("files-upload -F input.txt -S ${AGAVE_STORAGE_SYSTEM_ID} ./"))
    r.runagavecmd("LD_LIBRARY_PATH=/usr/local/lib "+
            "mpirun -np 4 /home/jovyan/FUNWAVE-TVD/src/funwave_vessel && "+
            "tar czf output.tgz output",
            "agave://${AGAVE_STORAGE_SYSTEM_ID}/input.txt")
    os.system("rm -fr output")
    os.system(repvar("jobs-output-get -S ${AGAVE_STORAGE_SYSTEM_ID} -N output-${JOB_ID}.tgz ${JOB_ID} output.tgz"))
    os.system("tar xzf output-${JOB_ID}.tgz")
    data = np.genfromtxt("output/eta_00010")
    fig = plt.figure(figsize=(12,12))
    pltres = plt.imshow(data[::-1,:])
    plt.show()

# Funwave needs to see a decimal point for
# the parameters that we are configuring. This
# format statement ensures that we write "1.0"
# and not "1".
def fmt(v):
    return "%.1f" % v

# When the plot button is clicked, regenerate the input file.
def plotBtn_clicked(a):
    setvar("WAVE_AMP="+fmt(waveAmplitude.value))
    setvar("WAVE_WID="+fmt(waveWidth.value))
    setvar("WAVE_XC="+fmt(waveXc.value))
    setvar("WAVE_YC="+fmt(waveYc.value))
    writefile("input.txt","""
!INPUT FILE FOR FUNWAVE_TVD
  ! NOTE: all input parameter are capital sensitive
  ! --------------------TITLE-------------------------------------
  ! title only for log file
TITLE = VESSEL
  ! -------------------HOT START---------------------------------
HOT_START = F
FileNumber_HOTSTART = 1
  ! -------------------PARALLEL INFO-----------------------------
  ! 
  !    PX,PY - processor numbers in X and Y
  !    NOTE: make sure consistency with mpirun -np n (px*py)
  !    
PX = 4
PY = 1
  ! --------------------DEPTH-------------------------------------
  ! Depth types, DEPTH_TYPE=DATA: from depth file
  !              DEPTH_TYPE=FLAT: idealized flat, need depth_flat
  !              DEPTH_TYPE=SLOPE: idealized slope, 
  !                                 need slope,SLP starting point, Xslp
  !                                 and depth_flat
DEPTH_TYPE = FLAT
DEPTH_FLAT = 10.0
  ! -------------------PRINT---------------------------------
  ! PRINT*,
  ! result folder
RESULT_FOLDER = output/

  ! ------------------DIMENSION-----------------------------
  ! global grid dimension
Mglob = 500
Nglob = 100

  ! ----------------- TIME----------------------------------
  ! time: total computational time/ plot time / screen interval 
  ! all in seconds
TOTAL_TIME = 10.0
PLOT_INTV = 1.0
PLOT_INTV_STATION = 50000.0
SCREEN_INTV = 1.0
HOTSTART_INTV = 360000000000.0

WAVEMAKER = INI_GAU
AMP = $WAVE_AMP
Xc = $WAVE_XC
Yc = $WAVE_YC
WID = $WAVE_WID

  ! -----------------GRID----------------------------------
  ! if use spherical grid, in decimal degrees
  ! cartesian grid sizes
DX = 1.0
DY = 1.0
  ! ----------------SHIP WAKES ----------------------------
VESSEL_FOLDER = ./
NumVessel = 2
  ! -----------------OUTPUT-----------------------------
ETA = T
U = T
V = T
""")

plotBtn.on_click(plotBtn_clicked)
runBtn.on_click(runBtn_clicked)


inputBox